this script plots the paper reduction of countries after a specific country is removed

In [23]:
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import geopandas as gp
import os
import funding.core

In [24]:
raw_path="../../data/nf_folder/Data/DerivedData/Derived/cntry_author_full.csv"
filter_path='../../data/nf_folder/Data/DerivedData/Derived/dep-FundExchange/pub_no_foreign_top20.csv'
geo_path="../../data/nf_folder/Data/AdditionalData/world_map_shp/ne_50m_admin_0_countries.shp"

In [10]:
raw_df=pd.read_csv(raw_path)
raw_df=raw_df.rename(columns={'aucntry':'cntry'})
raw_df.head()

,cntry,year,dis,count
0,Afghanistan,2009,3,1
1,Afghanistan,2009,33,1
2,Afghanistan,2009,35,1
3,Afghanistan,2009,44,2
4,Afghanistan,2009,47,1


In [4]:
filter_df=pd.read_csv(profile_path)
filter_df.head()

,author_distinct,dis,count,srce_cntry,year
0,Afghanistan,1,2,China,2017
1,Afghanistan,4,1,China,2017
2,Afghanistan,7,1,China,2017
3,Afghanistan,12,1,China,2017
4,Afghanistan,15,1,China,2017


In [11]:
def read_group(path,groupby,reset_col):
    df=pd.read_csv(path)
    df=df.rename(columns={'author_distinct':'cntry'})
    df=df.groupby(groupby)['count'].sum().reset_index(name=reset_col)
    return df

In [14]:
raw_df=read_group(raw_path,['cntry'],'raw')
filter_df=read_group(filter_path,['cntry','srce_cntry'],'filter')

In [26]:
meta=raw_df.merge(filter_df,on='cntry')
meta['p']=(meta['raw']-meta['filter'])/meta['raw']
meta['p']=meta['p'].apply(lambda x:np.around(x,2))

In [27]:
meta

,cntry,raw,srce_cntry,filter,p
0,Afghanistan,399,Australia,391,0.02
1,Afghanistan,399,Brazil,397,0.01
2,Afghanistan,399,Canada,398,0.00
3,Afghanistan,399,China,392,0.02
4,Afghanistan,399,EU,392,0.02
...,...,...,...,...,...
4115,Zimbabwe,2924,Sweden,2850,0.03
4116,Zimbabwe,2924,Switzerland,2810,0.04
4117,Zimbabwe,2924,Taiwan,2920,0.00
4118,Zimbabwe,2924,United Kingdom,2510,0.14


In [28]:
map_dir="../../data/nf_folder/Figs/dep-FundExchange/influncial_area"
world_geo=funding.core.read_geofile(geo_path)
for c in meta.srce_cntry.unique():

    fig, ax = plt.subplots(figsize=(8,6))
    df=meta[meta.srce_cntry==c]
    
    plot_data=world_geo.merge(df,left_on='WoS',right_on='cntry')
    plot_data=gp.GeoDataFrame(plot_data)
    fig,ax=funding.core.plot_worldmap(ax, fig, world_geo, plot_data, 'p','viridis_r',0.0,0.2,'')
    ax.margins(0)
    plot_path=os.path.join(map_dir, c+ '_map'+'.' + "pdf")
    plt.savefig(plot_path,bbox_inches='tight')
    plt.close()